# Neurons as logic gates


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

**Sigmoid function**


In [ ]:
#Define the sigmoid function
def sigmoid(x):
  """Sigmoid function"""
  return 1.0 / (1.0 + np.exp(-x))

In [1]:
#Plot the sigmoid function
vals = np.linspace(-10, 10, num=100, dtype=np.float32)
activation = sigmoid(vals)
fig = plt.figure(figsize=(12,6))
plt.plot(vals, activation)
plt.grid(True, which='both')
plt.axhline(y=0, color='k')
plt.axvline(x=0, color='k')
plt.yticks()
plt.ylim([-0.5, 1.5]);

NameError: ignored

**Sigmoid as OR, AND, NOR and NAND gate**


How to choose weights and bias to gate works properly. OR - if both inputs are 0 -> sig(z) = 0, if any of input is 1 -> sig(z) = 1

In [ ]:
def logic_gate(w1, w2, b):
  #helper to create logic gate functions
  #plug in values for weights and bias
  return lambda x1, x2: sigmoid(w1 * x1 + w2 * x2 + b)

def test_gate(gate):
  #helper function to test the weight function
  for a,b in (0,0), (0,1), (1,0), (1,1):
    print("{}, {}: {}".format(a, b, np.round(gate(a, b))))

In [ ]:
or_gate = logic_gate(20, 20, -10)
test_gate(or_gate)

0, 0: 0.0
0, 1: 1.0
1, 0: 1.0
1, 1: 1.0


AND - if both or one of inputs is 0 -> sig(z) = 0, if both of them are 1 -> sig(z) = 1

In [ ]:
and_gate = logic_gate(15, 15, -20)
test_gate(and_gate)

0, 0: 0.0
0, 1: 0.0
1, 0: 0.0
1, 1: 1.0


NOR - any of inputs cannot be 1 to return sig(z) = 1

In [ ]:
nor_gate = logic_gate(-20, -20, 10)
test_gate(nor_gate)

0, 0: 1.0
0, 1: 0.0
1, 0: 0.0
1, 1: 0.0


NAND - only if both inputs are 1 sig(z) = 0

In [ ]:
nand_gate = logic_gate(-20, -20, 30)
test_gate(nand_gate)

0, 0: 1.0
0, 1: 1.0
1, 0: 1.0
1, 1: 0.0


XOR gate is impossible to perform using single neuron so we are made to use something more complex:

In [ ]:
def xor_gate(a, b):
  c = or_gate(a, b)
  d = nand_gate(a, b)
  return and_gate(c, d)
test_gate(xor_gate)


0, 0: 0.0
0, 1: 1.0
1, 0: 1.0
1, 1: 0.0


**Exercise**

Provided below are the following:

Three weight matrices W_1, W_2 and W_3 representing the weights in each layer. The convention for these matrices is that each  Wi,j  gives the weight from neuron  i  in the previous (left) layer to neuron  j  in the next (right) layer.
A vector x_in representing a single input and a matrix x_mat_in representing 7 different inputs.
Two functions: soft_max_vec and soft_max_mat which apply the soft_max function to a single vector, and row-wise to a matrix.
The goals for this exercise are:

For input x_in calculate the inputs and outputs to each layer (assuming sigmoid activations for the middle two layers and soft_max output for the final layer.
Write a function that does the entire neural network calculation for a single input
Write a function that does the entire neural network calculation for a matrix of inputs, where each row is a single input.
Test your functions on x_in and x_mat_in.

In [ ]:
W_1 = np.array([[2,-1,1,4],[-1,2,-3,1],[3,-2,-1,5]])
W_1

In [ ]:
W_2 = np.array([[3,1,-2,1],[-2,4,1,-4],[-1,-3,2,-5],[3,1,1,1]])
W_3 = np.array([[-1,3,-2],[1,-1,-3],[3,-2,2],[1,2,1]])